# Setup

In [18]:
from json import loads
import pandas as pd
import os

# Main function

In [28]:
# Load the JSON file
YEAR="2025-2026-Q1"
json = loads(open(f"db/{YEAR}.json").read())

dic = dict()
for key, article in json["articles"]["mapValue"]["fields"].items():

    fields = article["mapValue"]["fields"]
    name = fields.get("name", {}).get("stringValue")

    a_type = fields.get("article_type", {}).get("integerValue")
    if a_type is None or a_type != "1":
        continue

    available = fields.get("available", {}).get("booleanValue")
    if not available:
        continue

    degree = fields.get("degree", {}).get("integerValue") or fields.get("degree", {}).get("doubleValue")

    if degree is None:
        degree = fields.get("degree", {}).get("doubleValue")

    price_out = fields.get("price_out", {}).get("doubleValue")

    if price_out is None:
        price_out = fields.get("price_out", {}).get("integerValue")

    volume = fields.get("format", {}).get("integerValue")

    if volume is None:
        volume = fields.get("format", {}).get("doubleValue")

    beer_type = fields.get("type", {}).get("stringValue")

    ratio =  (float(degree) * float(volume)) / float(price_out)

    dic[name] = {"Degree": float(degree), "Price": float(price_out), "Volume": float(volume), "Type": beer_type, "Ratio": float(ratio)}

# Create a DataFrame
df = pd.DataFrame(dic).T

# Sort the DataFrame by the ratio
df = df.sort_values(by=["Ratio", "Price", "Volume"], ascending=[False, True, False])
create_dir_if_absent(f"csv/{YEAR}")

out = df.copy()
out.insert(0, "Rank", range(1, len(df) + 1))
out.to_csv(f"csv/{YEAR}/ranker.csv")
len(out)

292

# Top 20, any type of beer

In [42]:
df[df["Volume"] < 75].head(20)

,Degree,Price,Volume,Type,Ratio
Kerel Kaishaku Fût,15.0,2.5,25.0,Blonde,150.0
Kerel Kaishaku,15.0,3.5,33.0,Blonde,141.428571
Kasteel triple,11.0,2.6,33.0,Blonde,139.615385
Bush 10,10.5,2.5,33.0,Blonde,138.6
Piraat,10.5,2.5,33.0,Ambrée,138.6
Bush 12 Fût,12.0,2.2,25.0,Ambrée,136.363636
Chimay rouge,9.0,2.2,33.0,Trappiste,135.0
Corne du bois des pendus quadruple,12.0,3.0,33.0,Ambrée,132.0
Queue de charrue triple,9.0,2.3,33.0,Blonde,129.130435
Gulden Draak,10.7,2.8,33.0,Brune,126.107143


# Top 20, blonde beers

In [51]:
blonde_beers = df[(df["Type"] == "Blonde") & (df["Volume"] < 75)]
blonde_beers.to_csv(f"csv/{YEAR}/blonde_ranker.csv")
blonde_beers.head(20)

,Degree,Price,Volume,Type,Ratio
Kerel Kaishaku Fût,15.0,2.5,25.0,Blonde,150.0
Kerel Kaishaku,15.0,3.5,33.0,Blonde,141.428571
Kasteel triple,11.0,2.6,33.0,Blonde,139.615385
Bush 10,10.5,2.5,33.0,Blonde,138.6
Queue de charrue triple,9.0,2.3,33.0,Blonde,129.130435
Corne du bois des pendus 10 triple,10.0,2.8,33.0,Blonde,117.857143
Duvel,8.5,2.4,33.0,Blonde,116.875
Triple plaisir (la),8.0,2.3,33.0,Blonde,114.782609
Carolus triple d'or,9.0,2.6,33.0,Blonde,114.230769
Jupiler Fût,5.4,1.2,25.0,Blonde,112.5


# Top 20, ambrées

In [43]:
amber_beers = df[(df["Type"] == "Ambrée") & (df["Volume"] < 75)]
amber_beers.to_csv(f"csv/{YEAR}/amber_ranker.csv")
amber_beers.head(20)

,Degree,Price,Volume,Type,Ratio
Piraat,10.5,2.5,33.0,Ambrée,138.6
Bush 12 Fût,12.0,2.2,25.0,Ambrée,136.363636
Corne du bois des pendus quadruple,12.0,3.0,33.0,Ambrée,132.0
Gulden Draak 9000,10.5,2.9,33.0,Ambrée,119.482759
Bon secours heritage,8.0,2.5,33.0,Ambrée,105.6
Troubadour magma,9.0,2.9,33.0,Ambrée,102.413793
Carolus ambrio,8.0,2.6,33.0,Ambrée,101.538462
Quintine ambrée,8.5,2.9,33.0,Ambrée,96.724138
Satan red,8.0,2.8,33.0,Ambrée,94.285714
Divine (la),8.5,3.0,33.0,Ambrée,93.5


# Top trappistes

In [50]:
trapist_beers = df[(df["Type"] == "Trappiste") & (df["Volume"] < 75)]
trapist_beers.to_csv(f"csv/{YEAR}/trapist_ranker.csv")
trapist_beers

,Degree,Price,Volume,Type,Ratio
Chimay rouge,9.0,2.2,33.0,Trappiste,135.0
Rochefort triple extra,9.0,2.4,33.0,Trappiste,123.75
Rochefort 10,11.3,3.1,33.0,Trappiste,120.290323
Rochefort 8,9.2,2.7,33.0,Trappiste,112.444444
Chimay blanche,8.0,2.4,33.0,Trappiste,110.0
Chimay bleue,9.0,3.0,33.0,Trappiste,99.0
Rochefort 6,7.5,2.6,33.0,Trappiste,95.192308
Chimay Verte (150),10.0,3.5,33.0,Trappiste,94.285714
Westmalle double,7.0,2.7,33.0,Trappiste,85.555556
Westmalle extra,4.8,2.1,33.0,Trappiste,75.428571


# Top 20, brunes

In [49]:
brown_beers = df[(df["Type"] == "Brune") & (df["Volume"] < 75)]
brown_beers.to_csv(f"csv/{YEAR}/brown_ranker.csv")
brown_beers.head(20)

,Degree,Price,Volume,Type,Ratio
Gulden Draak,10.7,2.8,33.0,Brune,126.107143
Kasteel donker,11.0,2.9,33.0,Brune,125.172414
Peak grand cru,10.5,2.8,33.0,Brune,123.75
Black albert,13.0,3.5,33.0,Brune,122.571429
Straffe Hendrik quadrupel,11.0,3.1,33.0,Brune,117.096774
Moinette brune,8.5,2.4,33.0,Brune,116.875
Malheur 12,12.0,3.4,33.0,Brune,116.470588
Saint Bernardus Prior 8,8.0,2.3,33.0,Brune,114.782609
Gauloise brune,8.1,2.4,33.0,Brune,111.375
Saint Bernardus Abr 12,10.0,3.0,33.0,Brune,110.0


# Top 20, fruitées

In [46]:
fruit_beers = df[(df["Type"] == "Fruitée") & (df["Volume"] < 75)]
fruit_beers.to_csv(f"csv/{YEAR}/fruit_ranker.csv")
fruit_beers.head(20)

,Degree,Price,Volume,Type,Ratio
Queue de charrue rouge,8.7,2.7,33.0,Fruitée,106.333333
Gauloise fruits rouges,8.2,2.6,33.0,Fruitée,104.076923
Frambush,8.5,2.8,33.0,Fruitée,100.178571
Delirium red Fût,8.0,2.0,25.0,Fruitée,100.0
Pêche Mel Bush Fût,8.0,2.0,25.0,Fruitée,100.0
Chouffe cherry Fût,8.0,2.0,25.0,Fruitée,100.0
Val dieu fruitee,9.0,3.2,33.0,Fruitée,92.8125
Barbar Rouge,8.0,2.9,33.0,Fruitée,91.034483
Kasteel red Fût,8.0,2.2,25.0,Fruitée,90.909091
Tête de mort red,8.2,3.0,33.0,Fruitée,90.2


# Top blanches

In [53]:
white_beers = df[(df["Type"] == "Blanche") & (df["Volume"] < 75)]
white_beers.to_csv(f"csv/{YEAR}/white_ranker.csv")
white_beers

,Degree,Price,Volume,Type,Ratio
Bon secours prestige,9.0,2.8,33.0,Blanche,106.071429
St Hubertus Blanche Fût,7.2,2.0,25.0,Blanche,90.0
Chouffe blanche,6.0,2.2,33.0,Blanche,90.0
Blanche de Bruxelles,4.5,2.0,33.0,Blanche,74.25
St bernardus witbier,5.5,2.5,33.0,Blanche,72.6
Boriner vice,6.0,2.9,33.0,Blanche,68.275862
Blanche de Namur,4.5,1.8,25.0,Blanche,62.5
Troublette,5.6,3.0,33.0,Blanche,61.6


# Top 75cl

In [47]:
big = df[df["Volume"]  == 75]
big.to_csv(f"csv/{YEAR}/75cl_ranker.csv")
big

,Degree,Price,Volume,Type,Ratio
Bush 12,12.0,7.0,75.0,Ambrée,128.571429
Lupulus Hibernatus,9.0,5.7,75.0,Brune,118.421053
Lupulus blonde,8.5,5.5,75.0,Blonde,115.909091
Lupulus brune,8.5,5.7,75.0,Brune,111.842105
Binchoise brune,7.7,5.3,75.0,Brune,108.962264
Chimay Blanche (Cinq Cents),8.0,5.6,75.0,Trappiste,107.142857
Moinette blonde,8.5,6.0,75.0,Blonde,106.25
Lupulus Organicus,8.5,6.0,75.0,Blonde,106.25
Westmalle triple,9.5,7.2,75.0,Trappiste,98.958333
Chouffe,8.0,6.2,75.0,Blonde,96.774194
